In [51]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
from langchain_core.runnables import RunnableParallel
from langchain_core.messages import HumanMessage, AIMessage
import warnings

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

True

In [52]:
GROQ_API_KEY = os.getenv("GROQQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"]="advanced-rag"
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY

## Query transformation

- Lần lượt lấy từng chunks thực hiện query translation (sinh ra các query làm nhiệm vụ tìm kiếm luật) (phuc) - chưa làm nhưng ở trong code có sẵn prompt (tạo prompt tiếng nhật)

In [53]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
# Template for generating multiple perspectives on a user question
template = """あなたはAI言語モデルアシスタントです。あなたのタスクは、与えられた契約文の正当性を証明するために関連する法律を検索するための5つの異なるバージョンの質問を生成することです。元の契約文に対して異なる視点からの質問を作成することで、距離ベースの類似検索の限界を克服し、より適切な法的情報を取得できるようにします。これらの代替質問を改行で区切って提供してください。

元の契約文: {contract_clause}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatGroq(temperature=0)   # thực chất là một model LLMs 
    | StrOutputParser()         # chuyển đổi output từ model LLMs thành dạng string
    | (lambda x: x.split("\n")) # tách các câu hỏi thành list các câu hỏi
) # kết quả trả về là list các câu hỏi

In [54]:
generate_queries.invoke({"contract_clause":"⑦ 酒気を帯びて勤務しないこと。"})

['1. ⑦ 飲酒後に勤務することは禁止されていますか？',
 '2. ⑦ 酔っ払いの状態で勤務することは契約違反とみなされますか？',
 '3. ⑦ 飲酒中に勤務することは契約上の問題となりますか？',
 '4. ⑦ 酒気を帯びた状態での勤務は、労働条件に違反することを意味しますか？',
 '5. ⑦ 飲酒後の酒気を帯びた勤務は、契約上禁止されている行為ですか？']

## Routing

- Mỗi chunks thực hiện routingLLM để chọn ra route phù hợp. Route là danh sách các vectorDB về luật (đã tạo). Sau đó LLMs duyệt lại các route đó và trả về các route liên quan. (phuc)

In [ ]:
# Danh sách các vectorDB
# 1 


In [ ]:
class VectorStore(BaseModel):
    (
        "A vectorstore contains information about symptoms, treatment"
        ", risk factors and other information about malaria, type 1 and"
        "type 2 diabetes and migraines"
    )

    query: str


class SearchEngine(BaseModel):
    """A search engine for searching other medical information on the web"""

    query: str


router_prompt_template = (
    "You are an expert in routing user queries to either a VectorStore, SearchEngine\n"
    "Use SearchEngine for all other medical queries that are not related to malaria, diabetes, or migraines.\n"
    "The VectorStore contains information on malaria, diabetes, and migraines.\n"
    'Note that if a query is not medically-related, you must output "not medically-related", don\'t try to use any tool.\n\n'
    "query: {query}"
)

llm = ChatGroq(model="llama3-70b-8192", temperature=0)
prompt = ChatPromptTemplate.from_template(router_prompt_template)
question_router = prompt | llm.bind_tools(tools=[VectorStore, SearchEngine])

## Retrieval

- Sau đó các route tức là các vectorDB (đã được embedding từ trước - có thể gộp lại của nhiều bộ vectorDB được thực chất là gộp các file pickle) thực hiện retrieval (có thể chromaDB hoặc FAISS đã làm) hoặc thực hiện lọc theo metadata hoặc kết hợp 2 phương pháp truy xuất cho từng chunk thu được docs liên quan của chunk đó. Và thông qua LLMs duyệt lại. (phuc)

In [ ]:
# Thực hiện sinh ra các metadata cần thiết cho việc tìm kiếm
# metadata có sẵn gồm: 
# - law_name: Tên đầy đủ của luật, giúp nhận diện nhanh chóng (ví dụ: "昭和二十六年法律第四十五号　社会福祉法").
# - law_number: Số hiệu chính thức của luật, thường bao gồm thông tin về năm ban hành (ví dụ: "昭和二十六年法律第四十五号").
# - enactment_year: Năm ban hành của luật, hữu ích để lọc tài liệu theo thời gian.
# - latest_revision_date: Ngày cập nhật hoặc sửa đổi mới nhất của luật, cho biết phiên bản hiện hành. (bỏ)
# - chapter_number: Số chương của luật, giúp phân loại nội dung theo cấu trúc chương.
# - chapter_name: Tên chương, mô tả chủ đề chính của phần đó (ví dụ: "地域福祉の推進").
# - division_number: Số mục trong chương nếu luật được chia nhỏ thành nhiều mục.
# - division_name: Tên phần, giúp hiểu rõ nội dung và phạm vi của phần đó (ví dụ: "共同募金").
# - article_number: Số điều của luật, dùng để định vị nội dung cụ thể trong văn bản (ví dụ: "120").
# - article_name: Tên điều, mô tả ngắn gọn nội dung hoặc chức năng của điều đó (ví dụ: "結果の公告").
# - category: Chủ đề hay danh mục của luật (ví dụ: "福祉" cho phúc lợi xã hội), hỗ trợ việc phân loại nội dung theo lĩnh vực.
# - keywords: Các từ khóa quan trọng liên quan đến nội dung của luật (ví dụ: ["共同募金", "配分", "公告", "寄附金", "準備金", "拠出"]), giúp tìm kiếm nhanh theo các khía cạnh cụ thể.
# - applicable_entities: Các đối tượng, cá nhân hoặc tổ chức mà luật áp dụng hoặc bị ảnh hưởng (ví dụ: ["共同募金会", "地方自治体", "寄附者"]).
# - reference_articles: Danh sách các điều luật hoặc phần liên quan khác được trích dẫn trong văn bản, giúp liên kết các quy định có liên quan (ví dụ: ["第118条", "第119条"]).
# - document_source: Nguồn gốc của văn bản, thông tin về cơ quan ban hành hoặc tổ chức chịu trách nhiệm về nội dung (ví dụ: "内閣府").

In [ ]:
#   "law_name": "昭和二十六年法律第四十五号　社会福祉法",
#   "law_number": "昭和二十六年法律第四十五号",
#   "enactment_year": 1951,
#   "latest_revision_date": "2023-06-15",# bỏ
#   "chapter_number": "第3章",
#   "chapter_name": "地域福祉の推進",
#   "division_number": "第2節",
#   "division_name": "共同募金",
#   "article_number": "120",
#   "article_name": "結果の公告",
# "document_source": "内閣府"

# - law_name（法律名）: 法律の正式名称を記述してください。（例: 昭和二十六年法律第四十五号　社会福祉法）
# - law_number（法律番号）: 法律の番号を記述してください。（例: 昭和二十六年法律第四十五号）
# - enactment_year（制定年）: 法律が制定された年を西暦で記述してください。（例: 1951）
# - latest_revision_date（最新改正日）: 直近の改正日を記述してください。（例: 2023-06-15）。情報がない場合は null を返してください。
# - chapter_number（章番号）: 章がある場合、その番号を記述してください。（例: 第3章）
# - chapter_name（章名）: 該当する章の名称を記述してください。（例: 地域福祉の推進）
# - division_number（節番号）: 節がある場合、その番号を記述してください。（例: 第2節）。ない場合は null を返してください。
# - division_name（節名）: 該当する節の名称を記述してください。（例: 共同募金）。ない場合は null を返してください。
# - article_number（条番号）: 条がある場合、その番号を記述してください。（例: 120）
# - article_name（条名）: 該当する条の名称を記述してください。（例: 結果の公告）
# from langchain.prompts import ChatPromptTemplate

# # Multi Query: Different Perspectives
# # Template for generating multiple perspectives on a user question
# template = """あなたはAI言語モデルアシスタントです。
# プロンプト（Metadata抽出用）
# 以下の法律文書から、指定されたメタデータを抽出し、JSON形式で出力してください。各項目の説明と例を参考にし、可能な限り正確な情報を提供してください。

# 出力形式（JSON）
# {
#   "category": "福祉",
#   "keywords": ["共同募金", "配分", "公告", "寄附金", "準備金", "拠出"],
#   "applicable_entities": ["共同募金会", "地方自治体", "寄附者"],
#   "reference_articles": ["第118条", "第119条"]
# }

# 抽出すべきメタデータ
# - category（カテゴリ）: 法律の分野を記述してください。（例: 福祉）。情報がない場合は null を返してください。
# - keywords（キーワード）: 重要なキーワードを配列で記述してください。（例: ["共同募金", "配分", "公告", "寄附金", "準備金", "拠出"]）
# - applicable_entities（適用対象）: 法律の適用対象となる機関や団体を記述してください。（例: ["共同募金会", "地方自治体", "寄附者"]）
# - reference_articles（関連条文）: 関連する他の条文のリストを記述してください。（例: ["第118条", "第119条"]）。情報がない場合は [] を返してください。

# 入力データ（法律文書）
# {document}
# このプロンプトを適用し、適切なメタデータを抽出してください。
# """
# prompt_gene_metadata = ChatPromptTemplate.from_template(template)
# # llm = ChatGroq(model="llama3-70b-8192", temperature=0)

# gene_metadatas = (
#     prompt_gene_metadata 
#     | ChatGroq(temperature=0)                       # thực chất là một model LLMs 
#     | StrOutputParser()         # chuyển đổi output từ model LLMs thành dạng string
#     | (lambda x: x.split("\n")) # tách các câu hỏi thành list các câu hỏi
# ) # kết quả trả về là list các câu hỏi

In [58]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from langchain.output_parsers import JsonOutputKeyToolsParser

# Định nghĩa mô hình đầu ra
class MetadataExtractor(BaseModel):
    category: Optional[str] = Field(
        None, description="法律の分野 (null nếu không có thông tin)"
    )
    keywords: List[str] = Field(
        default=[], description="重要なキーワードのリスト"
    )
    applicable_entities: List[str] = Field(
        default=[], description="適用対象となる機関や団体"
    )
    reference_articles: List[str] = Field(
        default=[], description="関連する他の条文のリスト"
    )

# Tạo prompt template
metadata_extraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        あなたはAI言語モデルアシスタントです。
        プロンプト（Metadata抽出用）
        以下の法律文書から、指定されたメタデータを抽出し、JSON形式で出力してください。
        必ずすべての項目を埋めてください。情報が不明な場合は「不明」や適切な推測を行ってください。
        """),
        ("human", "法律文書: {document}")
    ]
)

llm = ChatGroq(model="llama3-70b-8192", temperature=0)
# Kết hợp với LLM và định dạng output JSON
metadata_extractor_chain = metadata_extraction_prompt | llm.with_structured_output(
    MetadataExtractor
)

# Ví dụ dữ liệu đầu vào
document = "共同募金会の寄附金は第118条に基づき配分される。地方自治体も対象となる。"

# Thực thi pipeline
result = metadata_extractor_chain.invoke({"document": document})
print(result)


category=None keywords=['共同募金会', '寄附金', '第118条'] applicable_entities=['地方自治体'] reference_articles=['第118条']


In [59]:
metadata_extractor_chain.invoke({"document":"平成十五年法律第五十七号　個人情報の保護に関する法律/第六章　個人情報保護委員会/第二節　監督及び監視/第百四十九条　委員会の権限の行使の制限/委員会は、前三条の規定により個人情報取扱事業者等に対し報告若しくは資料の提出の要求、立入検査、指導、助言、勧告又は命令を行うに当たっては、表現の自由、学問の自由、信教の自由及び政治活動の自由を妨げてはならない。\n前項の規定の趣旨に照らし、委員会は、個人情報取扱事業者等が第五十七条第一項各号に掲げる者（それぞれ当該各号に定める目的で個人情報等を取り扱う場合に限る。）に対して個人情報等を提供する行為については、その権限を行使しないものとする。"})

MetadataExtractor(category='個人情報保護', keywords=['個人情報', '保護', '委員会'], applicable_entities=['個人情報保護委員会'], reference_articles=['第五十七条第一項各号'])

In [61]:
# Load full document from json file
import json

with open("./full_corpus_110225_with_metadata.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

print(len(docs))

5016


In [ ]:
# using tqdm
from tqdm import tqdm

for doc in tqdm(docs[:200]):
    result = metadata_extractor_chain.invoke({"document": doc["content"]})
    if result.category:
        doc["metadata"]["category"] = result.category
    if result.keywords:
        doc["metadata"]["keywords"] = result.keywords
    if result.applicable_entities:
        doc["metadata"]["applicable_entities"] = result.applicable_entities
    if result.reference_articles:
        doc["metadata"]["reference_articles"] = result.reference_articles

# for doc in docs[:200]: 
#     print(doc["metadata"])
#     print(doc["content"])
#     result = metadata_extractor_chain.invoke({"document": doc["content"]})
#     if result.category:
#         doc["metadata"]["category"] = result.category
#     if result.keywords:
#         doc["metadata"]["keywords"] = result.keywords
#     if result.applicable_entities:
#         doc["metadata"]["applicable_entities"] = result.applicable_entities
#     if result.reference_articles:
#         doc["metadata"]["reference_articles"] = result.reference_articles

In [77]:
new_docs = docs[:20]
for new_doc in new_docs:
    print(new_doc["metadata"])
    print(new_doc["content"])
    if "keywords" in new_doc["metadata"]:
        new_doc["metadata"]["keywords"] = ", ".join(new_doc["metadata"]["keywords"])
    if "applicable_entities" in new_doc["metadata"]:
        new_doc["metadata"]["applicable_entities"] = ", ".join(new_doc["metadata"]["applicable_entities"])
    if "reference_articles" in new_doc["metadata"]:
        new_doc["metadata"]["reference_articles"] = ", ".join(new_doc["metadata"]["reference_articles"])
    print("=====================================")

{'law_number': '大正十一年法律第七十号', 'law_name': '健康保険法', 'enactment_year': '令和6年12月2日 施行', 'chapter_number': '第一章', 'chapter_name': '総則', 'division_number': '', 'division_name': '', 'article_number': '第一条', 'article_name': '目的', 'document_source': 'https://laws.e-gov.go.jp/law/211AC0000000070', 'category': '健康保険法', 'keywords': '健, 康, 保, 険, ,,  , 業, 務, 災, 害, ,,  , 疾, 病, ,,  , 負, 傷, ,,  , 死, 亡, ,,  , 出, 産', 'applicable_entities': '労, 働, 者, ,,  , 被, 扶, 養, 者', 'reference_articles': '労, 働, 者, 災, 害, 補, 償, 保, 険, 法, （, 昭, 和, 二, 十, 二, 年, 法, 律, 第, 五, 十, 号, ）, 第, 七, 条, 第, 一, 項, 第, 一, 号'}
大正十一年法律第七十号　健康保険法/第一章　総則/第一条　目的/この法律は、労働者又はその被扶養者の業務災害（労働者災害補償保険法（昭和二十二年法律第五十号）第七条第一項第一号に規定する業務災害をいう。）以外の疾病、負傷若しくは死亡又は出産に関して保険給付を行い、もって国民の生活の安定と福祉の向上に寄与することを目的とする。
{'law_number': '大正十一年法律第七十号', 'law_name': '健康保険法', 'enactment_year': '令和6年12月2日 施行', 'chapter_number': '第一章', 'chapter_name': '総則', 'division_number': '', 'division_name': '', 'article_number': '第二条', 'article_name': '基本的理念', 'document_source': 'https://laws

In [76]:
# lưu new_docs vào file .json
with open("./new_docs.json", "w", encoding="utf-8") as f:
    json.dump(new_docs, f, ensure_ascii=False, indent=4)

In [78]:
# tổ chức thành dạng như sau:
# from langchain_core.documents import Document

# docs = [
#     Document(
#         page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
#         metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
#     ),
#     ...
# ]

# load new_docs
import json

with open("./new_docs.json", "r", encoding="utf-8") as f:
    new_docs = json.load(f)

from langchain_core.documents import Document

test_docs = []
for doc in new_docs:
    test_docs.append(
        Document(
            page_content=doc["content"],
            metadata=doc["metadata"]
        )
    )

In [101]:
import json

from langchain_core.documents import Document

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

with open("./new_docs.json", "r", encoding="utf-8") as f:
    new_docs = json.load(f)

from langchain_core.documents import Document

test_docs = []
for doc in new_docs:
    test_docs.append(
        Document(
            page_content=doc["content"],
            metadata=doc["metadata"]
        )
    )
len(test_docs)
vectorstore = Chroma.from_documents(test_docs, hf_embeddings)

from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="law_name",
        description="法律の正式名称。",
        type="string",
    ),
    AttributeInfo(
        name="law_number",
        description="法律の公式番号、制定年を含む。",
        type="string",
    ),
    AttributeInfo(
        name="enactment_year",
        description="法律の制定年。",
        type="integer",
    ),
    AttributeInfo(
        name="chapter_number",
        description="法律の章番号。",
        type="string",
    ),
    AttributeInfo(
        name="chapter_name",
        description="法律の章の名称。",
        type="string",
    ),
    AttributeInfo(
        name="division_number",
        description="法律の節番号（存在する場合）。",
        type="string",
    ),
    AttributeInfo(
        name="division_name",
        description="法律の節の名称。",
        type="string",
    ),
    AttributeInfo(
        name="article_number",
        description="法律の条番号。",
        type="string",
    ),
    AttributeInfo(
        name="article_name",
        description="法律の条の名称。",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="法律のカテゴリー。",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="法律に関連するキーワードのリスト。",
        type="long string",
    ),
    AttributeInfo(
        name="applicable_entities",
        description="法律が適用される対象。",
        type="long string",
    ),
    AttributeInfo(
        name="reference_articles",
        description="関連する条文のリスト。",
        type="long string",
    ),
    AttributeInfo(
        name="document_source",
        description="法律の発行元機関。",
        type="string",
    )
]

from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_groq import ChatGroq

document_content_description = "法律の内容"
llm = ChatGroq(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [100]:
retriever.invoke("2015年以降に施行される法律を教えてください")

OutputParserException: Parsing text
```json
{
    "query": "",
    "filter": "gt(enactment_year, 2014)"
}
```
 raised following error:
Unexpected token Token('COMMA', ',') at line 1, column 18.
Expected one of: 
	* LPAR
Previous tokens: [Token('CNAME', 'enactment_year')]

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## GraphRAG

- chunk + docs qua graphRAG => chunk, docs, generation, history (xây dựng tương tự file 8 ) (phuc)

## History + RAG

- chunk tiếp theo sẽ dùng retrieval tìm ra top 5 chunks liên quan nhất đến chunks hiện tại và qua LLMs đánh giá. Nếu thực sự liên quan thì sẽ lấy history của chunk trước bỏ vào để trả lời câu hiện tại. (phuc)